In [1]:
import os
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm
from bert_score import score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer
from IPython.display import display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from local_bert_scorer import LocalBERTScorer

In [2]:
MODEL_DIR = '/project/lt200301-edubot/Capstone-TamTanai/models'
LOG_DIR = '/project/lt200301-edubot/Capstone-TamTanai/logs'


def find_best_model(model_name):
    log_path = os.path.join(LOG_DIR, f'{model_name}.csv')
    log = pd.read_csv(log_path)
    step = int(log.iloc[log['eval_loss'].argmin()]['step'])
    return os.path.join(MODEL_DIR, model_name, f'checkpoint-{step}')

In [3]:
model_type = '/project/lt200301-edubot/bert_score/models/microsoft/deberta-xlarge-mnli'
bert_scorer = LocalBERTScorer(model_type=model_type, num_layers=40)

In [4]:
model_name = 'typhoon2-qwen2.5-7b-instruct-finetune-dataset-v1-exam-thailegal-with-v1+exam+thailegal-validation-set'
model_path = find_best_model(model_name)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [6]:
def run_evaluation(result_path):
    df = pd.read_csv(result_path)
    df['retrieval_result'] = df['retrieval_result'].apply(ast.literal_eval)
    answers = df['answer'].tolist()
    generation_results = df['generation_result'].tolist()
    
    precision, recall, f1 = bert_scorer.score(generation_results, answers, verbose=True, batch_size=8)
    #print('\nBert Score')
    print("average time:", df["processing_time"].mean())
    print(f'Precision: {precision.mean().item()}')
    print(f'Recall:    {recall.mean().item()}')
    print(f'F1:        {f1.mean().item()}')

    tokenizer.model_max_length = 1000000000
    rouge = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
    for answer, generation_result in tqdm(zip(answers, generation_results), total=len(answers)):
        scores = scorer.score(answer, generation_result)
        for k in rouge:
            rouge[k].append(scores[k].fmeasure)
    print('\nRouge')
    for k in rouge:
        print(f'{k}: {np.mean(rouge[k])}')

    # df['f1'] = f1
    # df = df.sort_values('f1')
    # for _, row in df.iloc[:].iterrows():
    #     print('\nF1:', row['f1'])
    #     print('Q:', row['question'])
    #     print("K:", df_dataset[df_dataset["question"]==row["question"]]["knowledges"].values[0])
    #     print('A:', row['answer'])
    #     print('R:', row['response'])
    #     print()

In [14]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=1.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/47 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 63.38 seconds, 6.52 sentences/sec
average time: 41.58832629483202
Precision: 0.9013776779174805
Recall:    0.9114153981208801
F1:        0.9058716893196106


100%|██████████| 413/413 [00:00<00:00, 460.77it/s]


Rouge
rouge1: 0.7848171197037843
rouge2: 0.7427846038785695
rougeL: 0.7577241159372605


In [7]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=1.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/94 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 53.80 seconds, 7.68 sentences/sec
average time: 41.8886224669348
Precision: 0.9006673693656921
Recall:    0.9108085632324219
F1:        0.9052121043205261


100%|██████████| 413/413 [00:00<00:00, 460.08it/s]


Rouge
rouge1: 0.7817684124473204
rouge2: 0.7398364636652752
rougeL: 0.7558139181995059


In [7]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=2.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/95 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 57.56 seconds, 7.17 sentences/sec
average time: 42.14883283552766
Precision: 0.8836186528205872
Recall:    0.8980758190155029
F1:        0.8902021050453186


100%|██████████| 413/413 [00:00<00:00, 448.96it/s]


Rouge
rouge1: 0.7450152049285323
rouge2: 0.6964247030512593
rougeL: 0.7111807973906524


In [8]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=2.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/95 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 46.15 seconds, 8.95 sentences/sec
average time: 42.15837123492151
Precision: 0.885330080986023
Recall:    0.9006953835487366
F1:        0.8923700451850891


100%|██████████| 413/413 [00:00<00:00, 446.86it/s]


Rouge
rouge1: 0.7480989957221991
rouge2: 0.7001792826337142
rougeL: 0.7155034117069333


In [8]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=5.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/98 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 48.24 seconds, 8.56 sentences/sec
average time: 47.83014587282268
Precision: 0.8643050193786621
Recall:    0.8772706985473633
F1:        0.8699427247047424


100%|██████████| 413/413 [00:00<00:00, 440.75it/s]


Rouge
rouge1: 0.6909773534285222
rouge2: 0.6277252366899784
rougeL: 0.6480458277777361


In [9]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=5.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/99 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 49.08 seconds, 8.41 sentences/sec
average time: 48.15134082694897
Precision: 0.8618673086166382
Recall:    0.8763154149055481
F1:        0.8681542277336121


100%|██████████| 413/413 [00:00<00:00, 467.29it/s]


Rouge
rouge1: 0.6866759308296444
rouge2: 0.6251654743555872
rougeL: 0.6453170531722123


In [9]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=10.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/100 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 49.47 seconds, 8.35 sentences/sec
average time: 70.30240731781967
Precision: 0.8457930684089661
Recall:    0.8546749949455261
F1:        0.8492355346679688


100%|██████████| 413/413 [00:00<00:00, 485.01it/s]


Rouge
rouge1: 0.6198210323718654
rouge2: 0.5370141859427581
rougeL: 0.5643786482992784


In [10]:
run_evaluation('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=10.csv')

calculating scores...
computing bert embedding.


  0%|          | 0/100 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/52 [00:00<?, ?it/s]

done in 50.26 seconds, 8.22 sentences/sec
average time: 70.35538534737096
Precision: 0.8457101583480835
Recall:    0.8535600900650024
F1:        0.8487105369567871


100%|██████████| 413/413 [00:00<00:00, 468.67it/s]


Rouge
rouge1: 0.6220551797256219
rouge2: 0.5377867230706476
rougeL: 0.566907793254058


In [10]:
df = pd.read_csv('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_end_to_end_n_documents=2.csv')
df['retrieval_result'] = df['retrieval_result'].apply(ast.literal_eval)
df

,question,answer,processing_time,domain_detection_result,domain_detection_probability,domain_detection_time_usage,retrieval_result,retrieval_reranking_score,retrieval_time_usage,generation_result,generation_time_usage
0,ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545 ...,'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพ...,43.779574,True,0.999809,0.006224,[พระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545\n\...,"[0.9999263140701885, 1.3184367302834308e-06]",0.623299,'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพ...,43.150021
1,หากลูกหนี้ไม่ได้กำหนดเวลาชำระหนี้ไว้ ลูกหนี้สา...,ลูกหนี้สามารถชำระหนี้ได้โดยพลัน,43.170337,True,0.999805,0.006054,[ประมวลกฎหมายแพ่งและพาณิชย์ - บรรพ 2 (หนี้) - ...,"[0.9999997654566101, 0.004663654952382039]",0.490520,ลูกหนี้สามารถชำระหนี้ได้โดยพลัน,42.673731
2,เครื่องหมายการค้าที่จดทะเบียนไว้ตามกฎหมายเดิมจ...,เครื่องหมายการค้าที่จดทะเบียนไว้ตามพระราชบัญญั...,42.373917,True,0.999814,0.006683,[พระราชบัญญัติเครื่องหมายการค้า (ฉบับที่ 3) พ....,"[0.9999929281049428, 0.29017562235469585]",0.412240,เครื่องหมายการค้าที่จดทะเบียนไว้ตามพระราชบัญญั...,41.954962
3,หากลูกหนี้ได้ชำระหนี้สินเต็มจำนวนแล้ว ศาลมีอำน...,ศาลมีอำนาจสั่งยกเลิกการล้มละลายหากหนี้สินของบุ...,42.257318,True,0.999817,0.006166,[พระราชบัญญัติล้มละลาย (ฉบับที่ 10) พ.ศ. 2561 ...,"[0.9840323465112041, 0.018511045990317374]",0.558598,ตามมาตรา 135 ของพระราชบัญญัติล้มละลาย (ฉบับที่...,41.692528
4,บัตรอิเล็กทรอนิกส์ตามนิยามในประมวลกฎหมายอาญาหม...,บัตรอิเล็กทรอนิกส์หมายถึงเอกสารหรือวัตถุอื่นใด...,42.746664,True,0.999812,0.006620,[ประมวลกฎหมายอาญา - ภาค 1 (บทบัญญัติทัว่ไป) - ...,"[0.9647888635784241, 7.571149413053982e-07]",0.125788,บัตรอิเล็กทรอนิกส์หมายความว่าเอกสารหรือวัตถุอื...,42.614180
...,...,...,...,...,...,...,...,...,...,...,...
408,สถานศึกษา สถานฝึกและอบรม และสถานแนะนำทางจิตที่...,จะเป็นสถานศึกษา สถานฝึกและอบรม และสถานแนะนำทาง...,42.545731,True,0.999813,0.005602,[พระราชบัญญัติศาลเยาวชนและครอบครัวและวิธีพิจาร...,"[0.9999257361871973, 0.00038446192991146674]",0.539736,จะถูกจัดการตามพระราชบัญญัติศาลเยาวชนและครอบครั...,42.000366
409,หากผู้ทำแผนไม่สามารถมาประชุมได้เพราะเหตุผลพิเศ...,หากผู้ทำแผนไม่สามารถมาประชุมได้เพราะเหตุผลพิเศ...,43.243825,True,0.999812,0.006647,[พระราชบัญญัติล้มละลาย (ฉบับที่ 10) พ.ศ. 2561 ...,"[0.9984929511366406, 0.0030396496140834887]",0.111255,หากผู้ทำแผนไม่สามารถมาประชุมได้เพราะเหตุผลพิเศ...,43.125839
410,หากบุคคลใดกรอกข้อมูลเท็จลงในแบบสำรวจ จะต้องรับ...,บุคคลดังกล่าวจะต้องระวางโทษจำคุกไม่เกินหกเดือน...,42.285835,True,0.999813,0.005737,[พระราชบัญญัติประกันสังคม พ.ศ. 2533 (ฉบับปรับป...,"[0.9999998115390829, 2.074190555355221e-06]",0.437796,ต้องระวางโทษจำคุกไม่เกินหกเดือน หรือปรับไม่เกิ...,41.842276
411,กรรมการค่าจ้างมีวาระการดำรงตำแหน่งกี่ปี?,กรรมการค่าจ้างมีวาระการดำรงตำแหน่งคราวละสองปี,41.917091,True,0.999811,0.005744,[พระราชบัญญัติคุ้มครองแรงงาน (ฉบับที่ 8) พ.ศ. ...,"[0.9999998130056972, 0.0031726828424851893]",0.155455,กรรมการค่าจ้างมีวาระการดำรงตำแหน่งคราวละสองปี,41.755864
